# Prohledávání hyperparametrů pro model MobileNetV2 nad datasetem CIFAR10

Tento notebook slouží k nalezení optimálních hyperparametrů nad datasetem CIFAR10 pro model MobileNetV2. Hyperparametry jsou hledány pro všechny varianty modelu (náhodně inicializovaný, předtrénovaný (doučení klasifikační hlavy) a předtrénovaný (doučení celého modelu)). Pro každou z variant jsou hledány hyperparametry pro normální trénink a trénink s destilací. 

K prohledávání je využito knihovny Optuna s algoritmem Hyperband. Nejlepší konfigurace je volena na základě F1-skóre, zkoušeno je 150 kombinací hyperparametrů pro každou variantu modelu. Při destilaci je využíváno předpočítaných logitů ze sešitu precompute_logits. 

## Import knihoven a základní nastavení

In [ ]:
from transformers import Trainer
import optuna
import torch
import math
import base
import os

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
dataset_part = base.get_dataset_part()

Resetování náhodného seedu pro replikovatelnost výsledků.

In [3]:
base.reset_seed()

Ověření dostupnosti GPU.

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


Načtení datasetu a aplikace základních transformací.

In [5]:
DATASET = "cifar10"

In [ ]:
transform = base.base_transforms()

test = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.TEST, transform=transform)
train = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.TRAIN, transform=transform)
eval = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.EVAL, transform=transform)

Základní konfigurace tréninku během prohledávání. Optuna nepracuje s epochami, ale s kroky. Níže je prováděn přepočet. 

Minimální délka tréninku jsou tři epochy, maximální deset epoch. Maximální počet kroků pro warm up je nastaven na 10 % první epochy.

In [8]:
num_epochs = 10
batch_size = 128

In [ ]:
data_length = len(train)
min_r = math.ceil(data_length/batch_size)*3
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

## Prohledávání s normálním tréninkem náhodně inicializovaného modelu

Definice hledaných hyperparametrů a jejich rozmezí. 

In [10]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny, běhy jsou vyhodnocovány pomocí algoritmu Hyperband, pro volbu další kombinace hyperparametrů je využito TPESampleru. 

In [11]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [12]:
base.reset_seed()

Konfigurace jednotlivých tréninků.

In [13]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/random_hp-search", logging_dir=f"~/logs/{DATASET}/random_hp-search", epochs=num_epochs, batch_size=batch_size)

Konfigurace trenéra pro jednotlivé tréninky. 

In [14]:
trainer = Trainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.get_random_init_mobilenet(10)
)
  

Nastavení prohledávání.

In [15]:
best_base_random = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base-random",
    n_trials=150
)

[I 2025-03-25 00:15:46,954] A new study created in memory with name: Base-random


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.665300,1.244537,0.560400,0.575059,0.560358,0.545970
2,1.013500,0.816005,0.709300,0.712499,0.708921,0.709204
3,0.745300,0.728064,0.750600,0.753604,0.749972,0.746851


[I 2025-03-25 00:23:20,801] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 5}


...

[I 2025-03-25 10:54:55,731] Trial 73 pruned. 


Trial 74 with params: {'learning_rate': 0.0004991039464163667, 'weight_decay': 0.005, 'warmup_steps': 26}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.618500,1.163697,0.589800,0.590743,0.588830,0.580064
2,1.006100,0.819795,0.705400,0.707158,0.705113,0.702661
3,0.774400,0.703550,0.755300,0.755213,0.754472,0.752016
4,0.613900,0.574439,0.798500,0.801630,0.798444,0.797811
5,0.491300,0.541583,0.816400,0.819531,0.816638,0.816287
6,0.379400,0.543027,0.817900,0.829390,0.817782,0.818520
7,0.264900,0.602446,0.813600,0.820169,0.814421,0.810013


[W 2025-03-25 11:05:03,181] Trial 74 failed with parameters: {'learning_rate': 0.0004991039464163667, 'weight_decay': 0.005, 'warmup_steps': 26} because of the following error: OSError(116, 'Stale file handle').
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/integrations/integration_utils.py", line 250, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2241, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2639, in _inner_training_loop
    self._maybe_log_save_evaluate(tr_loss, grad_norm, model, trial, epoch, ignore_keys_for_eval, start_time)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 3085, in 

OSError: [Errno 116] Stale file handle

In [ ]:
print(best_base_random)

In [14]:
base.reset_seed()

## Prohledávání s destilací do náhodně inicializovaného modelu
Konfigurace jednotlivých tréninků.

In [15]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/random-KD_hp-search", logging_dir=f"~/logs/{DATASET}/random-KD_hp-search",  remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí, rozšířeno o hyperparametry destilace.

In [16]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny. 

In [17]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace destilačního trenéra pro jednotlivé tréninky. 

In [18]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.get_random_init_mobilenet(10)
)
  

Nastavení prohledávání.

In [19]:
best_distill_random = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-03-25 13:20:13,187] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.229800,0.872347,0.569200,0.581125,0.569284,0.561281
2,0.767500,0.635119,0.712100,0.721079,0.712172,0.711780
3,0.596300,0.566114,0.753300,0.757952,0.752947,0.750231


[I 2025-03-25 13:24:52,947] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 28, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.148500,0.925519,0.539200,0.565401,0.538422,0.530601
2,0.783200,0.638121,0.713300,0.716016,0.713139,0.710264
3,0.623200,0.592369,0.734600,0.741206,0.733342,0.730470


[I 2025-03-26 12:25:38,335] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0006013711398701938, 'weight_decay': 0.008, 'warmup_steps': 9, 'lambda_param': 0.4, 'temperature': 5.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.137300,0.837469,0.592600,0.613968,0.592440,0.585716
2,0.744900,0.625158,0.715800,0.719280,0.715659,0.712727
3,0.587900,0.572607,0.743700,0.748696,0.743293,0.741153
4,0.492100,0.458909,0.805100,0.809126,0.805052,0.803396
5,0.419900,0.419990,0.826600,0.831498,0.826614,0.826707
6,0.353200,0.432605,0.818800,0.832337,0.818886,0.819065
7,0.296100,0.404011,0.838200,0.841311,0.838673,0.836259
8,0.246000,0.371450,0.848900,0.854292,0.849127,0.849125
9,0.206600,0.414665,0.835900,0.847119,0.835799,0.836011
10,0.179700,0.366399,0.850100,0.853568,0.850399,0.848966


[I 2025-03-26 12:38:01,748] Trial 149 finished with value: 0.8489659377203326 and parameters: {'learning_rate': 0.0006013711398701938, 'weight_decay': 0.008, 'warmup_steps': 9, 'lambda_param': 0.4, 'temperature': 5.5}. Best is trial 35 with value: 0.8607044045531035.


In [20]:
print(best_distill_random)

BestRun(run_id='35', objective=0.8607044045531035, hyperparameters={'learning_rate': 0.0006139968240256416, 'weight_decay': 0.007, 'warmup_steps': 4, 'lambda_param': 0.30000000000000004, 'temperature': 6.0}, run_summary=None)


In [21]:
base.reset_seed()

## Prohledávání s normálním tréninkem s doučením klasifikační hlavy předtrénovaného modelu
Konfigurace jednotlivých tréninků.

In [22]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/pretrained-head_hp-search", logging_dir=f"~/logs/{DATASET}/pretrained-head_hp-search", epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí.

In [ ]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [24]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace trenéra pro jednotlivé tréninky. 

In [27]:
trainer = Trainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.freeze_model(base.get_mobilenet(10))
)
  

config.json:   0%|          | 0.00/69.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

Nastavení prohledávání.

In [28]:
best_base_head = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base-head",
    n_trials=150
)

[I 2025-03-26 13:17:47,446] A new study created in memory with name: Base-head


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.315300,0.907658,0.712700,0.723356,0.712383,0.712754
2,0.836400,0.797999,0.735100,0.743364,0.733999,0.735253
3,0.762400,0.762252,0.744200,0.745174,0.743412,0.741890


[I 2025-03-26 13:20:17,766] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.887800,0.755003,0.738100,0.759657,0.738106,0.739319
2,0.719700,0.715052,0.752700,0.760723,0.752057,0.751523
3,0.691700,0.742330,0.745000,0.762037,0.744031,0.742473
4,0.681900,0.663716,0.768000,0.774258,0.767988,0.768298
5,0.668000,0.680603,0.763000,0.766968,0.762922,0.763262
6,0.660100,0.667803,0.767100,0.773936,0.766408,0.767948
7,0.643800,0.674470,0.769100,0.770992,0.769050,0.766154
8,0.633400,0.670348,0.768900,0.772851,0.768832,0.767885
9,0.629700,0.687548,0.764200,0.770523,0.764132,0.763901
10,0.615900,0.657127,0.772800,0.774076,0.772546,0.771583


[I 2025-03-27 04:09:01,945] Trial 148 finished with value: 0.7715827087741464 and parameters: {'learning_rate': 0.0025234679894924404, 'weight_decay': 0.0, 'warmup_steps': 18}. Best is trial 35 with value: 0.7718702742260117.


Trial 149 with params: {'learning_rate': 0.0026160813994285134, 'weight_decay': 0.0, 'warmup_steps': 19}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.888700,0.759756,0.737300,0.760032,0.737364,0.738452
2,0.721500,0.717489,0.751200,0.759859,0.750524,0.750222
3,0.693600,0.743203,0.744600,0.761801,0.743629,0.742229


[I 2025-03-27 04:11:31,898] Trial 149 pruned. 


In [29]:
print(best_base_head)

BestRun(run_id='35', objective=0.7718702742260117, hyperparameters={'learning_rate': 0.0024870786738035154, 'weight_decay': 0.009000000000000001, 'warmup_steps': 20}, run_summary=None)


In [30]:
base.reset_seed()

## Prohledávání s destilací s doučením klasifikační hlavy předtrénovaného modelu
Konfigurace jednotlivých tréninků.

In [31]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/pretrained-head-KD_hp-search", logging_dir=f"~/logs/{DATASET}/pretrained-head-KD_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí, rozšířeno o hyperparametry destilace.

In [32]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [33]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace destilačního trenéra pro jednotlivé tréninky. 

In [34]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.freeze_model(base.get_mobilenet(10))
)

Nastavení prohledávání.

In [35]:
best_distill_head = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-03-27 04:11:32,540] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.927800,0.710567,0.718900,0.732462,0.718512,0.719215
2,0.695700,0.671570,0.737700,0.745886,0.736877,0.737689
3,0.670100,0.661333,0.745200,0.745244,0.744454,0.743318


[I 2025-03-27 04:14:02,269] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 28, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.899500,0.704348,0.721000,0.735005,0.720627,0.721515
2,0.691600,0.668573,0.737900,0.745791,0.737124,0.737962
3,0.667700,0.659613,0.746600,0.746758,0.745844,0.744771


[I 2025-03-27 18:39:20,820] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.00036175490456492393, 'weight_decay': 0.002, 'warmup_steps': 22, 'lambda_param': 0.8, 'temperature': 4.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.886000,0.695763,0.724400,0.739898,0.723998,0.725209
2,0.685000,0.662976,0.740700,0.747817,0.740021,0.740660
3,0.663400,0.657246,0.748200,0.749375,0.747374,0.746632
4,0.655000,0.640252,0.755800,0.758613,0.755834,0.754828
5,0.648600,0.653766,0.743400,0.752654,0.743328,0.745109
6,0.645000,0.634844,0.758800,0.760390,0.758581,0.758213


[I 2025-03-27 18:44:15,205] Trial 149 pruned. 


In [36]:
print(best_distill_head)

BestRun(run_id='44', objective=0.7611287079618219, hyperparameters={'learning_rate': 0.00063155918393816, 'weight_decay': 0.009000000000000001, 'warmup_steps': 5, 'lambda_param': 0.6000000000000001, 'temperature': 4.0}, run_summary=None)


In [37]:
base.reset_seed()

## Prohledávání s normálním tréninkem s doučením předtrénovaného modelu
Konfigurace jednotlivých tréninků.

In [38]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/pretrained_hp-search", logging_dir=f"~/logs/{DATASET}/pretrained_hp-search", epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí.

In [39]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [40]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace trenéra pro jednotlivé tréninky. 

In [41]:
trainer = Trainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.get_mobilenet(10)
)
  

Nastavení prohledávání.

In [42]:
best_base_pretrained = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-03-27 18:44:15,803] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.465400,0.263481,0.909200,0.915397,0.908928,0.910108
2,0.138100,0.245159,0.921200,0.926270,0.921046,0.922330
3,0.070500,0.241737,0.926500,0.927986,0.926548,0.926458


[I 2025-03-27 18:48:00,751] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.414300,0.265083,0.911100,0.917484,0.910908,0.911401
2,0.137700,0.234567,0.922900,0.926557,0.922982,0.923564
3,0.069600,0.238316,0.929400,0.931321,0.929404,0.929504
4,0.039900,0.245360,0.936300,0.937160,0.936575,0.936407
5,0.023400,0.281287,0.932600,0.934035,0.932914,0.932195
6,0.011500,0.243560,0.942700,0.944244,0.942627,0.942886
7,0.005500,0.288150,0.938000,0.938962,0.938394,0.937605
8,0.002700,0.281722,0.941800,0.945409,0.942048,0.942251
9,0.001100,0.365279,0.934800,0.940302,0.934870,0.935369
10,0.000600,0.247187,0.947500,0.948062,0.947748,0.947483


[I 2025-03-28 15:38:04,314] Trial 148 finished with value: 0.9474828711537885 and parameters: {'learning_rate': 0.00029121261295695196, 'weight_decay': 0.01, 'warmup_steps': 6}. Best is trial 8 with value: 0.9546474828356917.


Trial 149 with params: {'learning_rate': 0.0002267720244026558, 'weight_decay': 0.01, 'warmup_steps': 0}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.408500,0.235990,0.918100,0.921483,0.917981,0.918525
2,0.126600,0.214763,0.929100,0.931925,0.929065,0.929614
3,0.059100,0.229177,0.929600,0.931048,0.929611,0.929720
4,0.032000,0.235309,0.935000,0.936381,0.935039,0.935149
5,0.017300,0.269431,0.933800,0.935966,0.933962,0.933891
6,0.007500,0.251777,0.942500,0.944039,0.942462,0.942724
7,0.003600,0.305077,0.934500,0.936185,0.934911,0.934131
8,0.002100,0.285053,0.942700,0.945562,0.942895,0.943079
9,0.000900,0.362798,0.932300,0.937638,0.932388,0.932737
10,0.000700,0.246884,0.947700,0.948363,0.947828,0.947719


[I 2025-03-28 15:50:30,732] Trial 149 finished with value: 0.947718598761725 and parameters: {'learning_rate': 0.0002267720244026558, 'weight_decay': 0.01, 'warmup_steps': 0}. Best is trial 8 with value: 0.9546474828356917.


In [43]:
print(best_base_pretrained)

BestRun(run_id='8', objective=0.9546474828356917, hyperparameters={'learning_rate': 0.00040842279473800845, 'weight_decay': 0.008, 'warmup_steps': 6}, run_summary=None)


In [44]:
base.reset_seed()

## Prohledávání s destilací s doučením předtrénovaného modelu
Konfigurace jednotlivých tréninků.

In [45]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/pretrained-KD_hp-search", logging_dir=f"~/logs/{DATASET}/pretrained-KD_hp-search",  remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí, rozšířeno o hyperparametry destilace.

In [46]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [47]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace destilačního trenéra pro jednotlivé tréninky. 

In [48]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    model_init = lambda: base.get_mobilenet(10)
)

Nastavení prohledávání.

In [49]:
best_distil_pretrained = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill",
    n_trials=150
)

[I 2025-03-28 15:50:31,280] A new study created in memory with name: Distill


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 24, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.396200,0.271806,0.907700,0.916947,0.907262,0.908197
2,0.197300,0.231288,0.930100,0.934120,0.930146,0.931004
3,0.156600,0.225188,0.932700,0.934921,0.932737,0.932686


[I 2025-03-28 15:54:16,137] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 28, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.425400,0.275556,0.908700,0.917378,0.908306,0.909287
2,0.198500,0.233298,0.930500,0.935370,0.930506,0.931398
3,0.156800,0.222667,0.933800,0.935504,0.933864,0.933911
4,0.136900,0.205400,0.940700,0.942808,0.940845,0.940818
5,0.128200,0.190829,0.949300,0.949988,0.949493,0.949372
6,0.122800,0.193715,0.948100,0.950061,0.948048,0.948407


[I 2025-03-29 13:10:09,680] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0003074746287433143, 'weight_decay': 0.006, 'warmup_steps': 9, 'lambda_param': 0.4, 'temperature': 4.0}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.367600,0.284001,0.906200,0.917131,0.905908,0.907322
2,0.197000,0.238982,0.928700,0.934220,0.928727,0.929790
3,0.157000,0.220615,0.934900,0.936435,0.935003,0.934949
4,0.138900,0.200463,0.942200,0.943372,0.942405,0.942269
5,0.128500,0.186519,0.949800,0.950317,0.949866,0.949829
6,0.122600,0.185698,0.950300,0.951754,0.950239,0.950577
7,0.119200,0.199788,0.946100,0.946894,0.946465,0.945954
8,0.117200,0.179723,0.951100,0.953511,0.951299,0.951453
9,0.115900,0.198068,0.941800,0.944650,0.941830,0.942149
10,0.114700,0.177481,0.952900,0.953299,0.952979,0.952929


[I 2025-03-29 13:22:38,712] Trial 149 finished with value: 0.9529290857790734 and parameters: {'learning_rate': 0.0003074746287433143, 'weight_decay': 0.006, 'warmup_steps': 9, 'lambda_param': 0.4, 'temperature': 4.0}. Best is trial 77 with value: 0.958376301709421.


In [50]:
print(best_distil_pretrained)

BestRun(run_id='77', objective=0.958376301709421, hyperparameters={'learning_rate': 0.00041679053444085474, 'weight_decay': 0.002, 'warmup_steps': 2, 'lambda_param': 0.30000000000000004, 'temperature': 4.5}, run_summary=None)


In [ ]:
print("Best random init training score: ", best_base_random)
print("Best random init distilation trianing score: ", best_distill_random)
print("Best pretrained (head only) training score: ", best_base_head)
print("Best pretrained distilation (head only) training score: ",best_distill_head)
print("Best pretrained training score: ", best_base_pretrained)
print("Best pretrained distilation training score: ", best_distil_pretrained)